In [2]:
import os
import json
import pickle
import numpy as np
import pandas as pd
import xarray as xr
import cv2
from datetime import datetime
from PIL import Image
from libtiff import TIFF
import matplotlib.pyplot as plt
from dea_tools.plotting import display_map, rgb, xr_animation

In [2]:
def find_timestamps(directory):
    """
    Returns
    -------
    timestamps: list of str
        A sorted list of strings taken as the first 23 characters in each filename (except PSScene_collection.json)
    """
    timestamps = set()
    for s in os.listdir(directory):
        timestamp = s[:23]
        timestamps.add(timestamp)
    timestamps.remove('PSScene_collection.json')
    timestamps = sorted(timestamps)
    return timestamps

In [3]:
def find_bboxs(directory, timestamps, tiff_prefix):
    """
    Returns
    -------
    bboxs: list of (list of float)
        The bounding box of each image using the local EPSG:32755 coordinate reference system (as opposed to the global EPSG:6933 CRS)
    """
    bboxs = []
    for timestamp in timestamps:
        filename = directory + timestamp + ".json"
        file = open(filename)
        metadata = json.loads(file.read())
        bbox = metadata['assets'][timestamp + tiff_prefix + "_tif"]['proj:bbox']  # epsg32755 (Local)
        # bbox = metadata['bbox']  # epsg6933 (Global)
        bboxs.append(bbox)
    return bboxs

In [4]:
def find_max_bbox(bboxs):
    """
    Returns
    -------
    max_bbox: list of float
    """
    bbox_stats = pd.DataFrame(bboxs).describe().loc[["min","max"]]
    max_bbox = [bbox_stats.loc["min",0], bbox_stats.loc["min",1], bbox_stats.loc["max",2], bbox_stats.loc["max",3]] 
    return max_bbox

In [5]:
# Not working for now. Gets the shape right, but the video looks like horizontal padding (vertical before rotation) is incorrectly distributed between left and right sides
def pad_band(band, bbox, max_bbox):
    """Pad the band with zeros if too small

    Parameters
    ----------
    band: 2d-array (x, y)
    bbox: list of float
    max_bbox: list of float
    """
    if bbox[0] > max_bbox[0]:
        required_padding = (bbox[0] - max_bbox[0]) / 3   # 3m pixel size
        left_buffer = np.zeros((band.shape[0], int(required_padding)))
        band = np.hstack((left_buffer, band))
    if bbox[1] > max_bbox[1]:
        required_padding = (bbox[1] - max_bbox[1]) / 3   
        upper_buffer = np.zeros((int(required_padding), band.shape[1]))
        band = np.vstack((upper_buffer, band))
    if bbox[2] < max_bbox[2]:
        required_padding = (max_bbox[2] - bbox[2]) / 3  
        right_buffer = np.zeros((band.shape[0], int(required_padding)))
        band = np.hstack((band, right_buffer))
    if bbox[3] < max_bbox[3]:
        required_padding = (max_bbox[3] - bbox[3]) / 3   
        lower_buffer = np.zeros((int(required_padding), band.shape[1]))
        band = np.vstack((band, lower_buffer))
    return band

def pad_image(image, bbox, max_bbox):
    """Apply padding to each of the bands in the image

    Parameters
    ----------
    image: 3d-array (band, x, y)
    bbox: list of float
    max_bbox: list of float
    
    """
    padded_image = [pad_band(band, bbox, max_bbox) for band in image]
    return np.array(padded_image)

In [6]:
# The alternative to padding
def extract_good_timestamps(timestamps, bboxs, max_bbox):
    """ 
    Extract just the timestamps where the bounding box coordinates match the maximum bounding box
    
    Returns
    -------
    timestamps: list of str
    """
    timestamps = [timestamp for timestamp, bbox in zip(timestamps, bboxs) if bbox == max_bbox]
    return timestamps

In [7]:
def load_images(directory, timestamps, tiff_prefix):
    """Load images without any preprocessing
    
    Returns
    -------
    images:list of 3d-array
        list of 3 dimensional arrays (band, x, y)
        There are 4 bands (red, green, blue, alpha)
    """
    images = []
    for timestamp in timestamps:
        filename = directory + timestamp + tiff_prefix + ".tif"
        tif = TIFF.open(filename) 
        image = tif.read_image()   # 4 bands: red, green, blue, alpha
        images.append(image)
    return images

In [8]:
def mask_image(image):
    """Assign np.nan to x,y coords where alpha is 0

    Parameters
    ----------
    image: 3d-array (band, x, y)
        4x bands (red, green, blue, alpha)

    Returns
    -------
    image: 3d-array (band, x, y)
        3x bands (red, green, blue)
    """
    mask = image[3]
    mask[mask == 0] = np.nan
    image[0]*=mask
    image[1]*=mask
    image[2]*=mask
    return image[:3]

In [9]:
def create_datetimes(timestamps):
    """
    Parameters
    ----------
    timestamps: list of str

    Returns
    -------
    datetimestamps: list of DateTime

    """
    datetimestamps = []
    for timestamp in timestamps:
        year = timestamp[0:4]
        month = timestamp[4:6]
        day = timestamp[6:8]
        hour = timestamp[9:11]
        minute = timestamp[11:13]
        second = timestamp[13:15]
        datetimestamp = datetime(int(year), int(month), int(day), hour=int(hour), minute=int(minute), second=int(second))
        datetimestamps.append(datetimestamp)
    return datetimestamps

In [10]:
def calc_truncated_columns(shape_planetscope, shape_sentinel):
    """Calculate the number of planetscape columns to truncate to match the sentinel aspect ratio"""
    ratio_sentinel = shape_sentinel[1]/shape_sentinel[0]
    keep = round(shape_planetscope[0]*ratio_sentinel)
    truncate = shape_planetscope[1] - keep
    return truncate

In [11]:
def create_lat_lon(bbox, shape):
    """ Create the latitudes and longitudes

    Parameters
    ----------
    bbox: list of float
    shape: list of int
    
    Returns
    -------
    x: 1d-array
    y: 1d-array
    """
    pixel_size = (bbox[2] - bbox[0])/shape[0], (bbox[3] - bbox[1])/shape[1]
    y = np.arange(bbox[0], bbox[2], pixel_size[0])
    x = np.arange(bbox[1], bbox[3], pixel_size[1])
    return x, y

In [12]:
# This function takes too long for a jupyter notebook so I have split it into separate cells below. Need to create a .py file if we want to run everything in one function like this.
def load_planetscope_3bands(directory, sentinel_shape, tiff_prefix):
    """
    Parameters
    ----------
    directory: string
        The location of the downloaded planetscope 3 band imagery, containing 3 files for each timestamp (xxx_3B_Visual_clip.tif, xxx_metadata.json, xxx.json)
        e.g. "../Planet/Farms/ARBO/74dbb359-bf0e-4445-a643-7423ef87edf7/PSScene/"
    sentinel_shape: (x,y)
        Final shape for each image to be resized into
    tiff_prefix: str
        "_3B_Visual_clip" or "_3B_AnalyticMS_SR_8b_clip" or "_3B_udm2_clip"

    Returns
    -------
    ds: xarray.Dataset
        All of the images in the directory get loaded into an xarray dataset
        Images that don't fill the bbox are padded with np.nan
        The EPSG:32755 "proj:bbox" is used to determine the outside coordinates
        The pixel size is determined by (max_coord - min_coord) / image_size
    """
    timestamps = find_timestamps(directory)
    bboxs = find_bboxs(directory, timestamps, tiff_prefix)
    max_bbox = find_max_bbox(bboxs)
    images = load_images(directory, timestamps, tiff_prefix)
    padded_images = [pad_image(image, bbox, max_bbox) for image, bbox in zip(images, bboxs)]
    image_array = np.array(padded_images)
    masked_images = np.array([mask_image(image) for image in image_array])
    truncated_columns = calc_truncated_columns(ds_sentinel["nbart_red"][0].shape, sentinel_shape)
    truncated_images = np.array([[band[truncated_columns:] for band in image] for image in masked_images])
    resized_images = np.array([[cv2.resize(band, sentinel_shape) for band in image] for image in truncated_images])
    rotated_images = np.array([[np.rot90(np.flip(band, axis=1)) for band in image] for image in resized_images])
    datetimestamps = create_datetimes(timestamps)
    x, y = create_lat_lon(max_bbox, sentinel_shape)
    transposed_images = rotated_images.transpose(1,0,2,3)
    ds_planetscope = xr.Dataset(
        {
            "nbart_red":(["time", "y", "x"], transposed_images[0]),
            "nbart_green":(["time", "y", "x"], transposed_images[1]),
            "nbart_blue":(["time", "y", "x"], transposed_images[2]),
        }, coords={
            "time": datetimestamps,
            "y": ("y", y),
            "x": ("x", x),
        },
    )
    return ds_planetscope

In [13]:
start = datetime.now()
print(start)

2024-04-16 15:04:46.430687


In [4]:
# This Sentinel data was generated and downloaded by running "sentinel_download.ipynb" on the digital Earth Australia Sandbox
with open('sentinel_arbo_4bands_2019-2024.pickle', 'rb') as handle:
    ds_sentinel = pickle.load(handle)
sentinel_shape = ds_sentinel['nbart_red'][0].shape
sentinel_shape

(660, 657)

In [15]:
# The directories containing the 3-band planetscope data
tiff_prefix = "_3B_Visual_clip"

# directory = "../Planet/Farms/ARBO/5e9dfe15-321f-49f2-9f64-1eb6a4e11d4c/PSScene/"
# outpath = 'planetscope_arbo_3bands_2020-2021.pickle'

directory = "../Planet/Farms/ARBO/74dbb359-bf0e-4445-a643-7423ef87edf7/PSScene/"
outpath = 'planetscope_arbo_3bands_2021-2024.pickle'

directory

'../Planet/Farms/ARBO/74dbb359-bf0e-4445-a643-7423ef87edf7/PSScene/'

In [16]:
timestamps = find_timestamps(directory)
timestamps[0]

'20210809_231254_09_2460'

In [17]:
bboxs = find_bboxs(directory, timestamps, tiff_prefix)
len(bboxs)

500

In [18]:
max_bbox = find_max_bbox(bboxs)
max_bbox

[685566.0, 6089898.0, 692232.0, 6096501.0]

In [19]:
# Can remove this if we get the padding working correctly
timestamps = [timestamp for timestamp, bbox in zip(timestamps, bboxs) if bbox == max_bbox]
len(timestamps)

312

In [20]:
images = load_images(directory, timestamps, tiff_prefix)

TIFFReadDirectory: Warning, Unknown field with tag 33550 (0x830e) encountered.
TIFFReadDirectory: Warning, Unknown field with tag 33922 (0x8482) encountered.
TIFFReadDirectory: Warning, Unknown field with tag 34735 (0x87af) encountered.
TIFFReadDirectory: Warning, Unknown field with tag 34737 (0x87b1) encountered.
TIFFReadDirectory: Warning, Unknown field with tag 33550 (0x830e) encountered.
TIFFReadDirectory: Warning, Unknown field with tag 33922 (0x8482) encountered.
TIFFReadDirectory: Warning, Unknown field with tag 34735 (0x87af) encountered.
TIFFReadDirectory: Warning, Unknown field with tag 34737 (0x87b1) encountered.
TIFFReadDirectory: Warning, Unknown field with tag 33550 (0x830e) encountered.
TIFFReadDirectory: Warning, Unknown field with tag 33922 (0x8482) encountered.
TIFFReadDirectory: Warning, Unknown field with tag 34735 (0x87af) encountered.
TIFFReadDirectory: Warning, Unknown field with tag 34737 (0x87b1) encountered.
TIFFReadDirectory: Warning, Unknown field with tag 3

In [21]:
# Not working for now. 
# images = [pad_image(image, bbox, max_bbox) for image, bbox in zip(images, bboxs)]

In [22]:
image_array = np.array(images, dtype=float)
image_array.shape

(312, 4, 2201, 2222)

In [23]:
masked_images = np.array([mask_image(image) for image in image_array])
masked_images.shape

(312, 3, 2201, 2222)

In [24]:
truncated_columns = calc_truncated_columns(masked_images[0][0].shape, sentinel_shape)
truncated_columns

31

In [25]:
# truncated_images = np.array([[band[truncated_columns:] for band in image] for image in masked_images])
truncated_images = np.array([[band[:, truncated_columns:] for band in image] for image in masked_images])
truncated_images.shape

(312, 3, 2201, 2191)

In [26]:
reversed_shape = sentinel_shape[1], sentinel_shape[0]
resized_images = np.array([[cv2.resize(band, reversed_shape) for band in image] for image in truncated_images])
resized_images.shape

(312, 3, 660, 657)

In [27]:
datetimestamps = create_datetimes(timestamps)
datetimestamps[0]

datetime.datetime(2021, 8, 11, 23, 15, 40)

In [28]:
x, y = create_lat_lon(max_bbox, sentinel_shape)
len(x), len(y)

(657, 660)

In [29]:
transposed_images = resized_images.transpose(1,0,2,3)
transposed_images.shape

(3, 312, 660, 657)

In [30]:
ds_planetscope = xr.Dataset(
    {
        "nbart_red":(["time", "y", "x"], transposed_images[0]),
        "nbart_green":(["time", "y", "x"], transposed_images[1]),
        "nbart_blue":(["time", "y", "x"], transposed_images[2]),
    }, coords={
        "time": datetimestamps,
        "y": ("y", y),
        "x": ("x", x),
    },
)
ds_planetscope = ds_planetscope.odc.assign_crs(crs='EPSG:32755') # Required for xr_animation
ds_planetscope

<xarray.Dataset> Size: 3GB
Dimensions:      (time: 312, y: 660, x: 657)
Coordinates:
  * time         (time) datetime64[ns] 2kB 2021-08-11T23:15:40 ... 2024-02-27...
  * y            (y) float64 5kB 6.856e+05 6.856e+05 ... 6.922e+05 6.922e+05
  * x            (x) float64 5kB 6.09e+06 6.09e+06 ... 6.096e+06 6.096e+06
    spatial_ref  int32 4B 32755
Data variables:
    nbart_red    (time, y, x) float64 1GB nan nan nan nan ... nan nan nan nan
    nbart_green  (time, y, x) float64 1GB nan nan nan nan ... nan nan nan nan
    nbart_blue   (time, y, x) float64 1GB nan nan nan nan ... nan nan nan nan

In [31]:
with open(outpath, 'wb') as handle:
    pickle.dump(ds_planetscope, handle, protocol=pickle.HIGHEST_PROTOCOL)
outpath

'planetscope_arbo_3bands_2021-2024.pickle'

In [32]:
end = datetime.now()
print(end-start)

0:10:25.204643


In [33]:
# Took 2 mins for unpadded 3 band planetscope 2020-2021 (98 images)
# Took 4 mins for padded 3 band planetscope 2020-2021 (156 images)
# Took 10 mins for unpadded 3 band planetscope 2021-2024 (312 images)